In [402]:
import numpy as np
import pandas as pd
from requests import get
from bs4 import BeautifulSoup

In [403]:
response = get('https://sfbay.craigslist.org/d/apartments-housing-for-rent/search/apa?availabilityMode=0&pets_dog=1')
html_soup = BeautifulSoup(response.text, 'html.parser')

posts = html_soup.find_all('li', class_ = 'result-row')
print(type(posts))
print(len(posts))

<class 'bs4.element.ResultSet'>
120


In [404]:
posts[1]

<li class="result-row" data-pid="7317532950">
<a class="result-image gallery" data-ids="3:00Q0Q_akCkakIMbCBz_0j20dl,3:00M0M_4bAjHc7x2y8z_0aO0aP,3:00404_v4WOzYDheCz_0j50dl,3:00Q0Q_RvruZTWgTZz_0CI0pO,3:00m0m_1TMa0CFQMiAz_0j70dl,3:00L0L_l8hwCdZAMFtz_0j90d0,3:00B0B_eIguf1s6OYsz_0e70aB,3:00k0k_cOjwVK94ilfz_0f30aA,3:00i0i_91YhjMtMOIez_0fS0ay,3:00C0C_blYO0uuUhvDz_0dU0aA,3:00f0f_hdjuqcmjEIVz_0jc0dx,3:00T0T_eDBj2Es97Qjz_0CI0pO,3:00J0J_2mfyKyyGScRz_0jb0d3,3:00h0h_hTDNnhZZ8Fsz_0jd0d9,3:00I0I_avMGG4yg7Bdz_0fw0aB" href="https://sfbay.craigslist.org/pen/apa/d/daly-city-stop-searching-this-is-the/7317532950.html">
<span class="result-price">$2,195</span>
</a>
<div class="result-info">
<span class="icon icon-star" role="button">
<span class="screen-reader-text">favorite this post</span>
</span>
<time class="result-date" datetime="2021-05-11 20:02" title="Tue 11 May 08:02:52 PM">May 11</time>
<h3 class="result-heading">
<a class="result-title hdrlnk" data-id="7317532950" href="https://sfbay.craigslist.

In [405]:
dog_price = []
dog_hood = []
dog_sqft = []
for i in np.arange(0, 3000, 120):
    response = get('https://sfbay.craigslist.org/d/apartments-housing-for-rent/search/apa?availabilityMode=0&pets_dog=1&s=' + str(i))
    html_soup = BeautifulSoup(response.text, 'html.parser')
    posts = html_soup.find_all('li', class_ = 'result-row')
    for k in np.arange(0, len(posts)):
        dog_price.append(int(posts[k].find('span', class_ = 'result-price').text.strip('$').replace(',', '')))
        
        if posts[k].find('span', class_ = 'result-hood') is None:
            dog_hood.append(None)
            print(i,k)
        elif posts[k].find('span', class_='result-hood') is not None:
            dog_hood.append(posts[k].find('span', class_ = 'result-hood').text.replace(' (','').replace(')',''))
        
        if posts[k].find('span', class_ = 'housing') is None:
            dog_sqft.append(None)
        elif len([x for x in posts[k].find('span', class_ = 'housing').text.split() if 'ft' in x]) == 0:
            dog_sqft.append(None)
        elif len([x for x in posts[k].find('span', class_ = 'housing').text.split() if 'ft' in x]) > 0:
            dog_sqft.append(int([x for x in posts[k].find('span', class_ = 'housing').text.split() if 'ft' in x][0].replace('ft2','')))

720 16
960 42
1920 25
2040 90
2160 22
2160 29
2160 48
2280 36
2760 47


In [406]:
cat_price = []
cat_hood = []
cat_sqft = []
for i in np.arange(0, 3000, 120):
    response = get('https://sfbay.craigslist.org/d/apartments-housing-for-rent/search/apa?availabilityMode=0&pets_cat=1&s=' + str(i))
    html_soup = BeautifulSoup(response.text, 'html.parser')
    posts = html_soup.find_all('li', class_ = 'result-row')
    for k in np.arange(0, len(posts)):
        cat_price.append(int(posts[k].find('span', class_ = 'result-price').text.strip('$').replace(',', '')))
        if posts[k].find('span', class_ = 'result-hood') is None:
            cat_hood.append(None)
            print(i,k)
        elif posts[k].find('span', class_='result-hood') is not None:
            cat_hood.append(posts[k].find('span', class_ = 'result-hood').text.replace(' (','').replace(')',''))
        
        if posts[k].find('span', class_ = 'housing') is None:
            cat_sqft.append(None)
        elif len([x for x in posts[k].find('span', class_ = 'housing').text.split() if 'ft' in x]) == 0:
            cat_sqft.append(None)
        elif len([x for x in posts[k].find('span', class_ = 'housing').text.split() if 'ft' in x]) > 0:
            cat_sqft.append(int([x for x in posts[k].find('span', class_ = 'housing').text.split() if 'ft' in x][0].replace('ft2','')))

720 108
1080 40
2160 108
2400 85
2520 22
2520 30
2520 51
2640 60


# Doggo

In [407]:
for i in range(len(dog_hood)):
    if dog_hood[i] != None:
        dog_hood[i] = dog_hood[i].title()

In [408]:
a = [i for i in range(len(dog_sqft)) if dog_sqft[i] == None]

for i in sorted(a, reverse=True):
    del dog_price[i]
    del dog_hood[i]
    del dog_sqft[i]

In [409]:
c = [x/y for x,y in zip(dog_price, dog_sqft)]

In [412]:
c.index(1347.5)
del c[1622]
del dog_hood[1622]
max(c)

7.656330749354005

In [413]:

test = {'Hood':dog_hood, 'Price/sqft':c}
dog = pd.DataFrame(test)
dog_count = dog.groupby(['Hood']).count().sort_values('Price/sqft', ascending=False).rename(columns= {'Price/sqft':'Count'})
dog_count

,Count
Hood,
Dublin / Pleasanton / Livermore,124
Sunnyvale,112
Fremont / Union City / Newark,105
Mountain View,100
Santa Clara,97
...,...
East Oakland,1
"San Francisco, Ca",1
"Colusa Ct,San Bruno, Ca",1


In [415]:
## All Price/sqft will be the median price/sqft of all listings for that hood

In [416]:
dog_df2 = dog.groupby(['Hood']).median().sort_values('Price/sqft', ascending=False)
dog_df2

,Price/sqft
Hood,
Mission,6.302632
Marina / Cow Hollow,5.897590
San Anselmo,5.583333
Russian Hill,5.189169
Tiburon / Belvedere,4.981176
...,...
Oakland,1.865812
"Colusa Ct,San Bruno, Ca",1.811364
Aptos,1.788909


In [438]:
dog_final = pd.merge(dog_count, dog_df2, on='Hood').rename(columns={'Price/sqft_x':'Count', 'Price/sqft_y':'Price/sqft'})
dog_final = dog_final.loc[dog_final['Count'] >= 25]
dog_final

,Count,Price/sqft
Hood,,
Dublin / Pleasanton / Livermore,124,2.939974
Sunnyvale,112,3.220618
Fremont / Union City / Newark,105,2.983759
Mountain View,100,3.390000
Santa Clara,97,2.952822
Concord / Pleasant Hill / Martinez,88,2.678413
Campbell,86,3.230769
Oakland Downtown,66,3.455704
San Jose Downtown,65,2.968493


# Cat

In [418]:
for i in range(len(cat_hood)):
    if cat_hood[i] != None:
        cat_hood[i] = cat_hood[i].title()

In [419]:
b = [i for i in range(len(cat_sqft)) if cat_sqft[i] == None]

for i in sorted(b, reverse=True):
    del cat_price[i]
    del cat_hood[i]
    del cat_sqft[i]

In [420]:
d = [x/y for x,y in zip(cat_price, cat_sqft)]

In [423]:
d.index(1347.5)
del d[1795]
del cat_hood[1795]
max(d)

7.777777777777778

In [424]:
test = {'Hood':cat_hood, 'Price/sqft':d}
cat = pd.DataFrame(test)
cat_count = cat.groupby(['Hood']).count().sort_values('Price/sqft', ascending=False).rename(columns = {'Price/sqft':'Count'})
cat_count

,Count
Hood,
Dublin / Pleasanton / Livermore,109
Sunnyvale,103
Mountain View,103
Fremont / Union City / Newark,103
Santa Clara,98
...,...
Laurel Hts / Presidio,1
San Carlos,1
East Oakland,1


In [426]:
cat_df2 = cat.groupby(['Hood']).median().sort_values('Price/sqft', ascending=False)
cat_df2

,Price/sqft
Hood,
Mission,6.302632
Tiburon / Belvedere,5.389874
Sunset / Parkside,5.200000
Castro / Upper Market,5.088481
Financial District,4.902834
...,...
Live In Portland!,1.855714
"Colusa Ct,San Bruno, Ca",1.811364
Aptos,1.788909


In [437]:
cat_final = pd.merge(cat_count, cat_df2, on='Hood').rename(columns={'Price/sqft_x':'Count', 'Price/sqft_y':'Price/sqft'})
cat_final = cat_final.loc[cat_final['Count'] >= 25]
cat_final

,Count,Price/sqft
Hood,,
Dublin / Pleasanton / Livermore,109,2.939974
Sunnyvale,103,3.198639
Mountain View,103,3.333973
Fremont / Union City / Newark,103,2.975026
Santa Clara,98,2.905325
Concord / Pleasant Hill / Martinez,86,2.651836
Campbell,82,3.230769
San Jose Downtown,81,2.968493
San Jose West,70,2.892812


# Both

In [439]:
Both = pd.merge(dog_final, cat_final, on = 'Hood').rename(columns={'Count_x':'Dog Count', 'Price/sqft_x':'Dog Price/sqft', 'Count_y':'Cat Count', 'Price/sqft_y':'Cat Price/sqft'})
Both

,Dog Count,Dog Price/sqft,Cat Count,Cat Price/sqft
Hood,,,,
Dublin / Pleasanton / Livermore,124,2.939974,109,2.939974
Sunnyvale,112,3.220618,103,3.198639
Fremont / Union City / Newark,105,2.983759,103,2.975026
Mountain View,100,3.390000,103,3.333973
Santa Clara,97,2.952822,98,2.905325
Concord / Pleasant Hill / Martinez,88,2.678413,86,2.651836
Campbell,86,3.230769,82,3.230769
Oakland Downtown,66,3.455704,54,3.568988
San Jose Downtown,65,2.968493,81,2.968493


In [440]:
diff = np.array(Both['Dog Price/sqft'] - Both['Cat Price/sqft'])
hoods = np.array(Both.index)
Differences = pd.DataFrame({'Hood':hoods, 'Difference':diff})
Differences

,Hood,Difference
0,Dublin / Pleasanton / Livermore,0.000000
1,Sunnyvale,0.021978
2,Fremont / Union City / Newark,0.008733
3,Mountain View,0.056027
4,Santa Clara,0.047497
5,Concord / Pleasant Hill / Martinez,0.026577
6,Campbell,0.000000
7,Oakland Downtown,-0.113284
8,San Jose Downtown,0.000000
9,San Jose South,-0.039464


In [441]:
print('Median price/sqft for Dogs is',np.median(c))
print('Median price/sqft for Cats is',np.median(d))

Median price/sqft for Dogs is 3.2015526148899838
Median price/sqft for Cats is 3.1736526946107784


In [442]:
print('Median price for Dogs is',np.median(dog_price))
print('Median price for Cats is',np.median(cat_price))

Median price for Dogs is 2482.0
Median price for Cats is 2440.0


In [443]:
print('Median sqft for Dogs is',np.median(dog_sqft))
print('Median sqft for Cats is',np.median(cat_sqft))

Median sqft for Dogs is 790.0
Median sqft for Cats is 781.0


## Inputted Manually 

In [444]:
Sunnyvale = [37.3688, -122.0363]
Oak_Grand = [37.8012, -122.2583]
Dub = [37.6604, -121.8758]
SantaClara = [37.3541, -121.9552]
Fremont = [37.5485, -121.9886]
SanMateo = [37.563, -122.3255]
MountainView = [37.3861, -122.0839]
SantaRosa = [38.4404, -122.7141]
SJWest = [37.3018, -121.9815]
SJDown = [37.3337, -121.8907]
SJSouth = [37.2761, -121.841]
Soma = [37.7836, -122.3899]
Antioch = [38.0049, -121.8058]
OaklandDown = [37.8033, -122.2711]
Concord = [37.9480, -122.0608]
Hayward = [37.6688, -122.0810]
Mission = [37.7599, -122.4148]
Berk = [37.8715, -122.2730]
Danville = [37.8216, -122]
Redwood = [37.4848, -122.2281]
Petaluma = [38.2324, -122.6367]
Emeryville = [37.8395, -122.2892]
Walnut = [37.9101, -122.0652]
Campbell = [37.2872, -121.95]
Milpitas = [37.4323, -121.8996]
Alameda = [37.7799, -122.2822]
Downtown = [37.7946, -122.3999]
Cupertino = [37.323, -122.0322]
Tender = [37.7847, -122.4145]
SJNorth = [37.3877, -121.9321]
Fair = [38.2492, -122.0405]
Foster = [37.5585, -122.2711]

Lat = [Dub[0], Sunnyvale[0], Fremont[0], MountainView[0], SantaClara[0], Concord[0], Campbell[0], OaklandDown[0],
      SJDown[0], SJSouth[0], SJWest[0], Redwood[0], Hayward[0], Oak_Grand[0], SantaRosa[0], 
      Soma[0], Danville[0], SanMateo[0], Walnut[0], Berk[0], Tender[0], Milpitas[0], Petaluma[0],
      Cupertino[0], Emeryville[0], Alameda[0], SJNorth[0], Mission[0], Fair[0], Downtown[0], Antioch[0], Foster[0]]

Long = [Dub[1], Sunnyvale[1], Fremont[1], MountainView[1], SantaClara[1], Concord[1], Campbell[1], OaklandDown[1],
      SJDown[1], SJSouth[1], SJWest[1], Redwood[1], Hayward[1], Oak_Grand[1], SantaRosa[1], 
      Soma[1], Danville[1], SanMateo[1], Walnut[1], Berk[1], Tender[1], Milpitas[1], Petaluma[1],
      Cupertino[1], Emeryville[1], Alameda[1], SJNorth[1], Mission[1], Fair[1], Downtown[1], Antioch[1], Foster[1]]

In [445]:
Differences['Lat'] = Lat
Differences['Long'] = Long
Differences

,Hood,Difference,Lat,Long
0,Dublin / Pleasanton / Livermore,0.000000,37.6604,-121.8758
1,Sunnyvale,0.021978,37.3688,-122.0363
2,Fremont / Union City / Newark,0.008733,37.5485,-121.9886
3,Mountain View,0.056027,37.3861,-122.0839
4,Santa Clara,0.047497,37.3541,-121.9552
5,Concord / Pleasant Hill / Martinez,0.026577,37.9480,-122.0608
6,Campbell,0.000000,37.2872,-121.9500
7,Oakland Downtown,-0.113284,37.8033,-122.2711
8,San Jose Downtown,0.000000,37.3337,-121.8907
9,San Jose South,-0.039464,37.2761,-121.8410


In [446]:
Differences['Difference'] = np.round(Differences['Difference'], decimals=3)
Differences

,Hood,Difference,Lat,Long
0,Dublin / Pleasanton / Livermore,0.000,37.6604,-121.8758
1,Sunnyvale,0.022,37.3688,-122.0363
2,Fremont / Union City / Newark,0.009,37.5485,-121.9886
3,Mountain View,0.056,37.3861,-122.0839
4,Santa Clara,0.047,37.3541,-121.9552
5,Concord / Pleasant Hill / Martinez,0.027,37.9480,-122.0608
6,Campbell,0.000,37.2872,-121.9500
7,Oakland Downtown,-0.113,37.8033,-122.2711
8,San Jose Downtown,0.000,37.3337,-121.8907
9,San Jose South,-0.039,37.2761,-121.8410


In [447]:
Differences.to_csv('Differences.csv')

In [448]:
len(dog_hood)

2518

In [449]:
len(cat_hood)

2443